In [1]:
from experiment_utils import ExperimentRun
import tensorflow as tf
import matplotlib.pyplot as plt
from models import build_generator, build_discriminator
from dataset import get_dataset
import os

# os.makedirs("results", exist_ok=True)

/Users/antranakhasi/Desktop/Projects/Origami model using CycleGAN/Origami-Model-using-CycleGAN/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ----------------------------
# 0️⃣ Hyperparameters
# ----------------------------
lambda_cyc, lambda_id = 5.0, 2.5
img_size = 256
batch_size = 2
num_epochs = 100
trainX_pattern = 'data/butterfly_real/*.jpeg'
trainY_pattern = 'data/butterfly_origami/*.jpg'

In [3]:
# ----------------------------
# 1️⃣ Generator & Discriminator
# ----------------------------
G = build_generator(image_size=img_size, n_blocks=9)
F = build_generator(image_size=img_size, n_blocks=9)
D_X = build_discriminator(image_size=img_size)
D_Y = build_discriminator(image_size=img_size)

In [4]:
# ----------------------------
# 2️⃣ Optimizer & Loss
# ----------------------------
G_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
D_X_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
D_Y_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)

mse = tf.keras.losses.MeanSquaredError()
L1 = lambda a, b: tf.reduce_mean(tf.abs(a - b))

In [5]:
#expereiment setup
params = {
    "lambda_cyc": lambda_cyc,
    "lambda_id": lambda_id,
    "img_size": img_size,
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "G_lr": 1e-4,
    "D_lr": 1e-4,
    "n_blocks": 9,
}

config = {
    **params,
    "trainX_pattern": trainX_pattern,
    "trainY_pattern": trainY_pattern,
}

exp = ExperimentRun(params=params, config=config)
logger = exp.get_logger()

# connect checkpoint manager
ckpt, ckpt_manager = exp.create_checkpoint_manager(
    G=G, F=F, D_X=D_X, D_Y=D_Y,
    G_optimizer=G_optimizer,
    D_X_optimizer=D_X_optimizer,
    D_Y_optimizer=D_Y_optimizer,
)

2025-11-17 19:31:30,419 - INFO - Experiment directory: experiments/run_20251117-193130


In [6]:
# ----------------------------
# 3️⃣ 1 training step
# ----------------------------
@tf.function
def train_step(real_x, real_y, lambda_cyc, lambda_id):
    with tf.GradientTape(persistent=True) as tape:

        fake_y = G(real_x, training=True)
        fake_x = F(real_y, training=True)

        cyc_x = F(fake_y, training=True)
        cyc_y = G(fake_x, training=True)

        same_x = F(real_x, training=True)
        same_y = G(real_y, training=True)

        D_X_real = D_X(real_x, training=True)
        D_X_fake = D_X(fake_x, training=True)
        D_Y_real = D_Y(real_y, training=True)
        D_Y_fake = D_Y(fake_y, training=True)

        G_GAN_loss = mse(tf.ones_like(D_Y_fake), D_Y_fake)
        F_GAN_loss = mse(tf.ones_like(D_X_fake), D_X_fake)

        D_X_loss = 0.5 * (mse(tf.ones_like(D_X_real), D_X_real) + mse(tf.zeros_like(D_X_fake), D_X_fake))
        D_Y_loss = 0.5 * (mse(tf.ones_like(D_Y_real), D_Y_real) + mse(tf.zeros_like(D_Y_fake), D_Y_fake))

        cycle_loss = L1(cyc_x, real_x) + L1(cyc_y, real_y)
        id_loss = L1(same_x, real_x) + L1(same_y, real_y)

        G_total_loss = G_GAN_loss + F_GAN_loss + lambda_cyc * cycle_loss + lambda_id * id_loss

    G_grads = tape.gradient(G_total_loss, G.trainable_variables + F.trainable_variables)
    D_X_grads = tape.gradient(D_X_loss, D_X.trainable_variables)
    D_Y_grads = tape.gradient(D_Y_loss, D_Y.trainable_variables)

    del tape
    G_optimizer.apply_gradients(zip(G_grads, G.trainable_variables + F.trainable_variables))
    D_X_optimizer.apply_gradients(zip(D_X_grads, D_X.trainable_variables))
    D_Y_optimizer.apply_gradients(zip(D_Y_grads, D_Y.trainable_variables))

    return G_total_loss, D_X_loss, D_Y_loss


In [ ]:

# ----------------------------
# 4️⃣ Dataset
# ----------------------------
trainX = get_dataset('data/butterfly_real/*.jpeg', batch_size=batch_size, img_size=img_size, augment=True)
trainY = get_dataset('data/butterfly_origami/*.jpg', batch_size=batch_size, img_size=img_size, augment=True)

In [ ]:

# ----------------------------
# 5️⃣ Training cycle
# ----------------------------
for epoch in range(num_epochs):
    
    logger.info(f"Epoch {epoch+1}/{num_epochs}")
    step = 0
    
    for real_x, real_y in zip(trainX, trainY):
        step += 1
        G_loss, DX_loss, DY_loss = train_step(real_x, real_y, lambda_cyc, lambda_id)
        if step % 50 == 0:
            logger.info(
                f"  step {step}: G={float(G_loss.numpy()):.3f} "
                f"DX={float(DX_loss.numpy()):.3f} DY={float(DY_loss.numpy()):.3f}"
            )
        loss_dict = {
            "G_total": G_loss,
            "D_X": DX_loss,
            "D_Y": DY_loss,
        }
        exp.log_losses(epoch + 1, step, loss_dict)

    exp.save_checkpoint()

    sample = next(iter(trainX))
    fake_y = G(sample, training=False)
    exp.save_sample_image(fake_y, name=f"epoch_{epoch+1:03d}_fake_y")

    if (epoch + 1) % 10 == 0:
        logger.info(
            "Epoch %d: G=%.3f DX=%.3f DY=%.3f",
            epoch + 1, float(G_loss.numpy()), float(DX_loss.numpy()), float(DY_loss.numpy())
        )
        sample = next(iter(trainX))
        fake_y = G(sample, training=False)
        plt.figure(figsize=(6, 3))
        plt.subplot(1, 2, 1); plt.imshow((sample[0] + 1) / 2); plt.title("Real X"); plt.axis("off")
        plt.subplot(1, 2, 2); plt.imshow((fake_y[0] + 1) / 2); plt.title("Fake Y"); plt.axis("off")
        plt.tight_layout()
        out_path = exp.results_dir / f"epoch_{epoch+1}.png"
        plt.savefig(out_path)
        plt.close()
        
print("\nGenerating final comparison on test image...")



In [ ]:
test_path = "data/NST_butterfly.jpg"
if os.path.exists(test_path):
    test_img = tf.io.read_file(test_path)
    test_img = tf.image.decode_jpeg(test_img, channels=3)
    test_img = tf.image.resize(test_img, [img_size, img_size])
    test_img = (tf.cast(test_img, tf.float32) / 127.5) - 1.0
    test_img = tf.expand_dims(test_img, 0)

    fake_y = G(test_img, training=False)
    fake_y = (fake_y[0] + 1.0) / 2.0

    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1); plt.imshow((test_img[0] + 1) / 2.0); plt.title("Original Image"); plt.axis("off")
    plt.subplot(1, 2, 2); plt.imshow(fake_y); plt.title("CycleGAN Output"); plt.axis("off")
    plt.tight_layout()
    plt.savefig("results/final_comparison.png")
    plt.close()
    print("✅ Final comparison saved at results/final_comparison.png")
else:
    print("⚠️ Skipped final comparison: test image not found at data/single_eval/test_img.jpg")
